# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# 02 – Attachments & Ingestion Demo (Drop-In Knowledge Mode)

This notebook demonstrates how the **Drop-In Knowledge Mode** runtime:

1. Works with **sessions** and basic conversational memory.
2. Accepts **attachments** via `AttachmentRef`.
3. Uses the `AttachmentIngestionService` to:
   - resolve attachment URIs to files,
   - load and split documents with Intergrax RAG components,
   - embed them,
   - store them in the vector store.

At this stage:

- We **do not** yet perform RAG retrieval when answering.
- The goal is to verify that:
  - attachments are correctly stored in the session,
  - ingestion runs without errors,
  - chunks are stored in the vector store,
  - ingestion details are visible in `debug_trace`.

In [13]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

## 1. Initialize the Drop-In Knowledge Mode runtime

In this section we:

1. Create an **in-memory session store** – suitable for notebooks and tests.
2. Instantiate an **LLM adapter** using Ollama + LangChain.
3. Configure:
   - `IntergraxEmbeddingManager` (Ollama embeddings),
   - `IntergraxVectorstoreManager` (Chroma as a vector store).
4. Create a `RuntimeConfig` and `DropInKnowledgeRuntime` instance.

We intentionally keep RAG turned off for now – we only care about ingestion,
not retrieval or context building.


In [14]:
# 1) In-memory session store (no external DB, good for experiments)
from intergrax.globals.settings import GLOBAL_SETTINGS
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.rag.embedding_manager import EmbeddingManager
from intergrax.rag.vectorstore_manager import VSConfig, VectorstoreManager
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import DropInKnowledgeRuntime
from intergrax.runtime.drop_in_knowledge_mode.ingestion.attachments import FileSystemAttachmentResolver
from intergrax.runtime.drop_in_knowledge_mode.ingestion.ingestion_service import AttachmentIngestionService
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from datetime import datetime
from pprint import pprint
from pathlib import Path

from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager


session_manager = SessionManager(
    storage=InMemorySessionStorage()
)

# 2) LLM adapter based on Ollama + LangChain
llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

# 3) High-level runtime configuration
config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
    enable_user_profile_memory=True,
)

# 4) Vector store connection (same collection as ingestion)
embedding_manager = EmbeddingManager(provider="ollama")

vectorstore_cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs_ingestion",    
)

# 5) Ingestion service configuration
ingestion_service = AttachmentIngestionService(
        resolver=FileSystemAttachmentResolver(),
        embedding_manager=embedding_manager,
        vectorstore_manager=VectorstoreManager(
            config=vectorstore_cfg
        )
)

runtime = DropInKnowledgeRuntime(
    config=config,    
    session_manager=session_manager,
    ingestion_service=ingestion_service,
)

print("Runtime initialized at", datetime.utcnow().isoformat())

[intergraxVectorstoreManager] Initialized provider=chroma, collection=intergrax_docs_ingestion
[intergraxVectorstoreManager] Existing count: 144
Runtime initialized at 2025-12-27T08:24:06.658709


C:\Users\XPS\AppData\Local\Temp\ipykernel_18984\662763891.py:58: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print("Runtime initialized at", datetime.utcnow().isoformat())


## 2. Prepare an AttachmentRef for a local project document

Now we want to simulate what happens when a user **attaches a file** in a chat UI.

In this example, we will:

- point to a local file (for example `PROJECT_STRUCTURE.md` at the repo root),
- wrap it in an `AttachmentRef`,
- let the runtime handle ingestion of this attachment.

Important notes:

- We use `Path(...).resolve().as_uri()` to generate a proper `file://` URI that
  the `FileSystemAttachmentResolver` can understand.
- In a real application, URIs could also be:
  - `s3://...`,
  - `db://attachments/<id>`,
  - or any other scheme supported by a custom `AttachmentResolver`.


In [15]:
# Adjust this path so that it points to a real file in your Intergrax repository.
# For this demo, we assume PROJECT_STRUCTURE.md is located at the repo root.
from intergrax.llm.messages import AttachmentRef


project_root_file = Path("../../PROJECT_STRUCTURE.md").resolve()

if not project_root_file.exists():
    raise FileNotFoundError(f"Expected demo file does not exist: {project_root_file}")

attachment_uri = project_root_file.as_uri()

attachment = AttachmentRef(
    id="project-structure-md",
    type="markdown",
    uri=attachment_uri,
    metadata={
        "description": "Intergrax project structure document",
        "scope": "intergrax-docs-demo",
    },
)

print("AttachmentRef prepared:")
print("  id       :", attachment.id)
print("  type     :", attachment.type)
print("  uri      :", attachment.uri)
print("  metadata :", attachment.metadata)


AttachmentRef prepared:
  id       : project-structure-md
  type     : markdown
  uri      : file:///D:/Projekty/intergrax/PROJECT_STRUCTURE.md
  metadata : {'description': 'Intergrax project structure document', 'scope': 'intergrax-docs-demo'}


## 3. First runtime call with an attachment

We now:

1. Create a `RuntimeRequest` with:
   - `user_id`,
   - `session_id`,
   - a user message,
   - a list containing our `AttachmentRef`.
2. Call `runtime.ask(request_1)`.
3. Inspect:
   - the model's answer,
   - the `RouteInfo`,
   - the `debug_trace["ingestion"]` field.

`debug_trace["ingestion"]` should contain:

- an entry per ingested attachment,
- number of generated chunks,
- how many vectors were stored,
- metadata such as source path and session identifiers.


In [16]:
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest


user_id = "demo-user-attachments"
session_id = "demo-session-attachments-001"

request_1 = RuntimeRequest(
    user_id=user_id,
    session_id=session_id,
    message=(
        "I am attaching a document that describes the Intergrax project "
        "structure. Please ingest it and confirm when you're ready."
    ),
    attachments=[attachment],
    metadata={"source": "02_attachments_ingestion_demo"},
)

answer_1 = await runtime.run(request_1)

print("=== ANSWER 1 ===")
print(answer_1.answer)

print("\n=== ROUTE INFO ===")
pprint(answer_1.route)

print("\n=== DEBUG TRACE: INGESTION ===")
pprint(answer_1.debug_trace.get("ingestion"))


[intergraxVectorstoreManager] Upserting 144 items (dim=1536) to provider=chroma...
[intergraxVectorstoreManager] Upsert complete. New count: 144
=== ANSWER 1 ===
I've retrieved the attached document, which appears to be `PROJECT_STRUCTURE.md`. I'm now familiar with the structure of the Intergrax project, including its components, modules, and key responsibilities.

What would you like me to help with regarding this project structure? Do you have specific questions or areas where you'd like clarification?

=== ROUTE INFO ===
RouteInfo(used_rag=False,
          used_websearch=False,
          used_tools=False,
          used_user_profile=False,
          used_user_longterm_memory=False,
          strategy='llm_with_session_attachments',
          extra={'attachments_chunks': 6, 'used_attachments_context': True})

=== DEBUG TRACE: INGESTION ===
[{'attachment_id': 'project-structure-md',
  'attachment_type': 'markdown',
  'metadata': {'session_id': 'demo-session-attachments-001',
         

## 4. Inspect the session state (messages and attachments)

Next, we want to verify that the **session store** correctly persisted:

- the user message,
- the assistant response,
- and the attachment references.

We will:

1. Load the session by `session_id`.
2. Inspect:
   - basic session metadata,
   - all `SessionMessage` objects,
   - their `attachments` and `metadata` fields.


In [17]:
session = await session_manager.get_session(session_id)
messages = await session_manager.get_history(session_id=session_id)

print("=== SESSION AFTER FIRST REQUEST ===")
print("Session ID :", session.id)
print("User ID    :", session.user_id)
print("Messages   :", len(messages))
print("Created at :", session.created_at)
print("Updated at :", session.updated_at)

print("\n=== MESSAGES ===")
for idx, msg in enumerate(messages, start=1):
    print(f"[{idx}] role={msg.role!r}, created_at={msg.created_at}")
    print(f"     content={msg.content!r}")
    if msg.attachments:
        print(f"     attachments ({len(msg.attachments)}):")
        for a in msg.attachments:
            print(f"       - id={a.id!r}, type={a.type!r}, uri={a.uri!r}")
    if msg.metadata:
        print(f"     metadata keys: {list(msg.metadata.keys())}")


=== SESSION AFTER FIRST REQUEST ===
Session ID : demo-session-attachments-001
User ID    : demo-user-attachments
Messages   : 2
Created at : 2025-12-27 08:24:06.671216+00:00
Updated at : 2025-12-27 08:24:16.529255+00:00

=== MESSAGES ===
[1] role='user', created_at=2025-12-27T08:24:11.281744+00:00
     content="I am attaching a document that describes the Intergrax project structure. Please ingest it and confirm when you're ready."
[2] role='assistant', created_at=2025-12-27T08:24:16.529255+00:00
     content="I've retrieved the attached document, which appears to be `PROJECT_STRUCTURE.md`. I'm now familiar with the structure of the Intergrax project, including its components, modules, and key responsibilities.\n\nWhat would you like me to help with regarding this project structure? Do you have specific questions or areas where you'd like clarification?"


## 5. Inspect the LLM-facing chat history

The runtime internally converts `ChatSession.messages` into a list of
`ChatMessage` objects that are passed to the LLM adapter.

Here we directly call the internal helper `_build_chat_history(session)` to:

- verify how the conversation history looks before the next model call,
- confirm that messages are preserved in order,
- see how roles and content are exposed to the LLM.

Note:
- Attachments are not included directly in `ChatMessage` at this stage.
  They are used by the ingestion pipeline and RAG, not embedded in raw text.


In [18]:
from intergrax.llm.messages import ChatMessage


history = await runtime._history_layer._build_chat_history(session)

print("=== BUILT CHAT HISTORY ===")
print("History length:", len(history))
for idx, msg in enumerate(history, start=1):
    assert isinstance(msg, ChatMessage)
    print(f"[{idx}] role={msg.role!r}, created_at={msg.created_at}")
    print(f"     content={msg.content!r}")


=== BUILT CHAT HISTORY ===
History length: 2
[1] role='user', created_at=2025-12-27T08:24:11.281744+00:00
     content="I am attaching a document that describes the Intergrax project structure. Please ingest it and confirm when you're ready."
[2] role='assistant', created_at=2025-12-27T08:24:16.529255+00:00
     content="I've retrieved the attached document, which appears to be `PROJECT_STRUCTURE.md`. I'm now familiar with the structure of the Intergrax project, including its components, modules, and key responsibilities.\n\nWhat would you like me to help with regarding this project structure? Do you have specific questions or areas where you'd like clarification?"


## 6. Second runtime call in the same session (no new attachments)

To mirror a typical ChatGPT-like workflow:

1. First message: user uploads a document and asks the system to ingest it.
2. Second message: user refers to that document and asks for a summary.

Here we:

- reuse the same `user_id` and `session_id`,
- send a second message without additional attachments,
- inspect the answer, route info, and debug trace.

At this stage, the answer will not yet use RAG retrieval. However:

- the ingestion has already populated the vector store,
- the session has accumulated multiple messages,
- the runtime is ready for the next step: **context builder + RAG**.


In [19]:
request_2 = RuntimeRequest(
    user_id=user_id,
    session_id=session_id,
    message=(
        "Great. In the previous message I attached the project structure file. "
        "Please summarize the main components of the Intergrax framework as "
        "described in that document."
    ),
    attachments=[],  # no new attachments; we rely on the existing session context
    metadata={"source": "02_attachments_ingestion_demo_second_call"},
)

answer_2 = await runtime.run(request_2)

print("=== ANSWER 2 ===")
print(answer_2.answer)

print("\n=== ROUTE INFO 2 ===")
pprint(answer_2.route)

print("\n=== DEBUG TRACE 2: ATTACHMENTS ===")
pprint(answer_2.debug_trace.get("attachments"))
print("attachments_chunks:", answer_2.debug_trace.get("attachments_chunks"))


=== ANSWER 2 ===
Based on the `PROJECT_STRUCTURE.md` document, here's a summary of the main components of the Intergrax framework:

1. **Domain: LLM instructions / Framework bundle**:
	* Contains all Python modules from the package directory
	* Provides a single source of truth for the Intergrax framework
2. **Key Responsibilities:**
	* Navigation through the bundle using the MODULE MAP and INDEX
	* Rules for proposing changes to ensure preservation of existing architecture, naming, and conventions
3. **Components:**
	* `main.py`: Entry point for the Intergrax framework, responsible for executing core functionality
	* `mcp\__init__.py`: Multimedia Processing module (downloads videos, transcribes audio, extracts frames)
	* `intergrax\openai\__init__.py`: OpenAI adapter initialization and configuration
	* `intergrax\runtime\drop_in_knowledge_mode\engine\runtime_state.py`: RuntimeState class definition for managing runtime pipeline state

Additionally, the document mentions several other 

In [22]:
await runtime.print_usage_runs()

runs: 2
Run #1
  ts_utc     : 2025-12-27T08:24:16.529255+00:00
  run_id     : run_908399f0a4f449dd9b4856def74769da
  session_id : demo-session-attachments-001
  user_id    : demo-user-attachments
LLMUsageReport(run_id=run_908399f0a4f449dd9b4856def74769da)
Total:
  calls        : 1
  input_tokens : 1093
  output_tokens: 66
  total_tokens : 1159
  duration_ms  : 4905
  errors       : 0
By provider/model:
  - LLMProvider.OLLAMA:llama3.1:latest: calls=1 in=1093 out=66 total=1159 ms=4905 err=0
Entries (registration order):
  - core_adapter [LLMProvider.OLLAMA:llama3.1:latest] (LangChainOllamaAdapter)
      calls=1 in=1093 out=66 total=1159 ms=4905 err=0 instance_id=2764292431488
Run #2
  ts_utc     : 2025-12-27T08:24:23.798516+00:00
  run_id     : run_5d271e64518243a1a57e4a596a506738
  session_id : demo-session-attachments-001
  user_id    : demo-user-attachments
LLMUsageReport(run_id=run_5d271e64518243a1a57e4a596a506738)
Total:
  calls        : 1
  input_tokens : 1377
  output_tokens: 301
